In [330]:
#!g1.1
import pandas as pd
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import xavier_uniform_
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# from torchtext.data import BucketIterator, TabularDataset, Iterator, Field, LabelField

root=''
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# device = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', device)

df_train = pd.read_csv(root + 'fun/train.csv')
# df_train.head()

Using PyTorch version: 1.4.0  Device: cuda


In [331]:
#!g1.1
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')
tokenizer.add_special_tokens({'unk_token': '[UNK]', 'bos_token': '[BOS]', 'eos_token': '[EOS]', 'pad_token': '[PAD]'})

3

In [282]:
#!g1.1
df_train = df_train.dropna()
df_train = df_train.reset_index(drop=True)
input_ids = []
attention_masks = []
sentences = df_train['text']
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                     
                        add_special_tokens = True,
                        max_length = 80,          
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df_train['label'])
train_dataset = TensorDataset(input_ids, attention_masks, labels)

batch_size = 64

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [283]:
#!g1.1
df_val = pd.read_csv('fun/test.csv')
df_val = df_val.dropna()
df_val = df_val.reset_index(drop=True)
input_ids = []
attention_masks = []
sentences = df_val['text']
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                     
                        add_special_tokens = True,
                        max_length = 80,          
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df_val['label'])
val_dataset = TensorDataset(input_ids, attention_masks, labels)

batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [284]:
#!g1.1
model = GPT2ForSequenceClassification.from_pretrained(
    "sberbank-ai/rugpt3medium_based_on_gpt2", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
).to(device)
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of the model checkpoint at sberbank-ai/rugpt3medium_based_on_gpt2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/rugpt3medium_based_on_gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [287]:
#!g1.1
from transformers import get_linear_schedule_with_warmup
import numpy as np
import time
import datetime

epochs = 1
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 4e-5,
                  eps = 1e-8
                )
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 500,
                                            num_training_steps = total_steps)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [286]:
#!g1.1
import wandb

# 1. Start a new run
wandb.init(project='bert-jokes')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rototo (use `wandb login --relogin` to force relogin)


In [297]:
#!g1.1
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []
total_t0 = time.time()
step = 0
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()        

        output = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels)
        loss = output[0]
        logits = output[1]

        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        scheduler.step()
        step += 1
        wandb.log({"train_loss": total_train_loss / step})

    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():        
            output = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels)
            loss = output[0]
            logits = output[1]
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 1 ========
Training...
  Batch    40  of  3,929.    Elapsed: 0:00:45.
  Batch    80  of  3,929.    Elapsed: 0:01:26.
  Batch   120  of  3,929.    Elapsed: 0:02:07.
  Batch   160  of  3,929.    Elapsed: 0:02:49.
  Batch   200  of  3,929.    Elapsed: 0:03:30.
  Batch   240  of  3,929.    Elapsed: 0:04:11.
  Batch   280  of  3,929.    Elapsed: 0:04:52.
  Batch   320  of  3,929.    Elapsed: 0:05:33.
  Batch   360  of  3,929.    Elapsed: 0:06:15.
  Batch   400  of  3,929.    Elapsed: 0:06:56.
  Batch   440  of  3,929.    Elapsed: 0:07:37.
  Batch   480  of  3,929.    Elapsed: 0:08:18.
  Batch   520  of  3,929.    Elapsed: 0:09:00.
  Batch   560  of  3,929.    Elapsed: 0:09:41.
  Batch   600  of  3,929.    Elapsed: 0:10:22.
  Batch   640  of  3,929.    Elapsed: 0:11:03.
  Batch   680  of  3,929.    Elapsed: 0:11:45.
  Batch   720  of  3,929.    Elapsed: 0:12:26.
  Batch   760  of  3,929.    Elapsed: 0:13:07.
  Batch   800  of  3,929.    Elapsed: 0:13:48.
  Batch   840  of

In [332]:
#!g1.1
def eval_model(model, loader):
    all_logits = []
    all_true = []
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in loader:

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():        
            output = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels)
            loss = output[0]
            logits = output[1]
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        all_logits.extend(list(logits))
        all_true.extend(list(label_ids))

        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(loader)
    print("Accuracy: {0:.2f}".format(avg_val_accuracy))
    all_preds = []
    for i in range(len(all_logits)):
        all_preds.append(F.softmax(torch.tensor(all_logits[i]), dim=-1)[1].item())
    return all_preds, all_true

In [301]:
#!g1.1
all_preds, all_true = eval_model(model, validation_dataloader)

Accuracy: 0.94


In [302]:
#!g1.1
from sklearn.metrics import classification_report
print(classification_report(all_true, (np.array(all_preds) > 0.5).astype(int), digits=5))

              precision    recall  f1-score   support

           0    0.94760   0.92423   0.93577     30897
           1    0.92606   0.94889   0.93734     30897

    accuracy                        0.93656     61794
   macro avg    0.93683   0.93656   0.93655     61794
weighted avg    0.93683   0.93656   0.93655     61794



In [303]:
#!g1.1
torch.save(model, 'classification/gpt_fun.pt')

In [343]:
#!g1.1
df_gold = pd.read_csv('fun/assessed.csv')
df_gold = df_gold.dropna()
df_gold = df_gold.drop(955, axis=0)
df_gold = df_gold.reset_index(drop=True)
input_ids = []
attention_masks = []
sentences = df_gold['text']
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                     
                        add_special_tokens = True,
                        max_length = 80,          
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt')
    input_ids.append(encoded_dict['input_ids'].long())
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df_gold['label'])
gold_dataset = TensorDataset(input_ids, attention_masks, labels)

batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
gold_dataloader = DataLoader(
            gold_dataset, # The validation samples.
            sampler = SequentialSampler(gold_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [344]:
#!g1.1
from sklearn.metrics import classification_report
model = torch.load('classification/gpt_fun.pt')
all_preds, all_true = eval(model, gold_dataloader)
print(classification_report(all_true, (np.array(all_preds) > 0.5).astype(int), digits=5))

Accuracy: 0.90
              precision    recall  f1-score   support

           0    0.93231   0.87410   0.90227       977
           1    0.87187   0.93103   0.90048       899

    accuracy                        0.90139      1876
   macro avg    0.90209   0.90257   0.90138      1876
weighted avg    0.90335   0.90139   0.90141      1876



In [312]:
#!g1.1
all_preds, all_true = eval(model2, gold_dataloader)
print(classification_report(all_true, (np.array(all_preds) > 0.5).astype(int), digits=5))

Error in callback <function _WandbInit._resume_backend at 0x7fb807901ef0> (for pre_run_cell):
Accuracy: 0.52
              precision    recall  f1-score   support

           0    0.56555   0.15623   0.24483     30897
           1    0.51051   0.87999   0.64616     30897

    accuracy                        0.51811     61794
   macro avg    0.53803   0.51811   0.44549     61794
weighted avg    0.53803   0.51811   0.44549     61794

Error in callback <function _WandbInit._pause_backend at 0x7fb807c4a710> (for post_run_cell):


In [ ]:
#!g1.1
